<a href="https://colab.research.google.com/github/czhang2718/LLGM/blob/main/notebooks/base_deepseek_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers
!pip3 install torch
!pip3 install huggingface_hub
!pip3 install flash_attn
!pip3 install accelerate
!pip3 install datasets
!pip3 install evaluate
!pip3 install numpy
!pip3 install peft
!pip3 install bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
cache_name = ".depsek"
dataset_name = "AnkitAI/CategorizedTextReviews"

In [3]:
from transformers import AutoModel, AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from huggingface_hub import snapshot_download, notebook_login
import torch
from datasets import load_dataset
import numpy as np
import evaluate
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [4]:
notebook_login()

In [5]:
snapshot_dir = snapshot_download(repo_id=model_name, cache_dir=cache_name)
print(snapshot_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

configuration_deepseek.py:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

tokenization_deepseek_fast.py:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

.depsek/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Instruct/snapshots/e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11


In [6]:
model_base = AutoModel.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
import sys
sys.path.append(".depsek/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Instruct/snapshots/")
!touch {snapshot_dir}/__init__.py
!ls {snapshot_dir}

config.json			   model-00002-of-000004.safetensors  README.md
configuration_deepseek.py	   model-00003-of-000004.safetensors  tokenization_deepseek_fast.py
generation_config.json		   model-00004-of-000004.safetensors  tokenizer_config.json
__init__.py			   modeling_deepseek.py		      tokenizer.json
model-00001-of-000004.safetensors  model.safetensors.index.json


In [8]:
from e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11 import modeling_deepseek

In [9]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)
config.hidden_size = 100
print(config.model_type)

deepseek_v2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
model_classify = modeling_deepseek.DeepseekV2ForSequenceClassification(config)
model_classify.model = model_base

In [11]:
model_modules = [(n, type(m)) for n, m in model_classify.named_modules()]
print(*model_modules, sep='\n')

Streaming output truncated to the last 5000 lines.
('model.layers.12.mlp.experts.12.up_proj', <class 'torch.nn.modules.linear.Linear'>)
('model.layers.12.mlp.experts.12.down_proj', <class 'torch.nn.modules.linear.Linear'>)
('model.layers.12.mlp.experts.12.act_fn', <class 'torch.nn.modules.activation.SiLU'>)
('model.layers.12.mlp.experts.13', <class 'transformers_modules.deepseek-ai.DeepSeek-Coder-V2-Lite-Instruct.e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11.modeling_deepseek.DeepseekV2MLP'>)
('model.layers.12.mlp.experts.13.gate_proj', <class 'torch.nn.modules.linear.Linear'>)
('model.layers.12.mlp.experts.13.up_proj', <class 'torch.nn.modules.linear.Linear'>)
('model.layers.12.mlp.experts.13.down_proj', <class 'torch.nn.modules.linear.Linear'>)
('model.layers.12.mlp.experts.13.act_fn', <class 'torch.nn.modules.activation.SiLU'>)
('model.layers.12.mlp.experts.14', <class 'transformers_modules.deepseek-ai.DeepSeek-Coder-V2-Lite-Instruct.e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11.modeling_deep

In [12]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    target_modules=["q_proj", "kv_a_proj_with_mqa", "kv_b_proj", "o_proj", 'gate_proj', 'up_proj', 'down_proj'],
    inference_mode=False,
    modules_to_save=["score"],
)

In [13]:
model_classify = get_peft_model(model_classify, peft_config)

In [14]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

In [16]:
dataset = load_dataset(dataset_name)

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [18]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [19]:
metric = evaluate.load("accuracy")

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    optim="adamw_bnb_8bit",
    auto_find_batch_size=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

In [22]:
trainer = Trainer(
    model=model_classify,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()